In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/home/nico/Projets/kaggle_competitions/playground-series-s4e8')
from variables import * 


In [ ]:
train = pd.read_parquet('/home/nico/Projets/kaggle_competitions/playground-series-s4e8/data/train.parquet')
train[TARGET_COL] = train[TARGET_COL].map(dict_label_target)
train.head()

## FEATURES ENGINEERING WITH TARGET ENCODING

In [ ]:
def get_dict_target_encoding(df,column,target) :

    dict_target = df[[column,target]].groupby([column],dropna=False).mean()[target].to_dict()
    
    return(dict_target)

In [ ]:
dict_target_encoding = {}
for cat_col in CAT_COLS :
    train[cat_col] = train[cat_col].fillna('NA') # to simplify target encoding
    dict_target_encoding[cat_col] = get_dict_target_encoding(train,cat_col,TARGET_COL)

In [ ]:
for col in CAT_COLS : 
    train[col] = train[col].map(dict_target_encoding[col])
train.head()

## XGBOOST 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.metrics import matthews_corrcoef

In [ ]:
X_train,X_test = train_test_split(train,train_size=0.25,random_state=42)

In [ ]:
FEATURES_COL = CONT_COLS + CAT_COLS

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1]
}

# param_grid = {
#     'max_depth': [ 5],
#     'learning_rate': [ 0.01],
#     'subsample': [ 0.7]
# }

In [ ]:
xgb_model = xgb.XGBClassifier(random_state = 42)
grid_search = GridSearchCV(xgb_model, param_grid, cv=2, scoring='accuracy',verbose=3)

In [ ]:
grid_search.fit(X_train[FEATURES_COL], X_train[TARGET_COL])

In [ ]:
# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
bst_estimator = grid_search.best_estimator_

In [ ]:
preds = bst_estimator.predict(X_test[FEATURES_COL])
score = matthews_corrcoef(X_test[TARGET_COL], preds)
print('MCC:', score)

## Submission

In [ ]:
test = pd.read_parquet('/home/nico/Projets/kaggle_competitions/playground-series-s4e8/data/test.parquet')

In [ ]:
for cat_col in CAT_COLS :
    test[cat_col] = test[cat_col].fillna('NA')
    test[cat_col] = test[cat_col].map(dict_target_encoding[cat_col]).fillna(np.mean(list(dict_target_encoding[cat_col].values())) )

In [ ]:
for cont_col in CONT_COLS :
    test[cont_col] = test[cont_col].fillna(test[cont_col].dropna().mean())

In [ ]:
test['class'] = bst_estimator.predict(test[FEATURES_COL])
test['class'] = test['class'].map(dict_target_label)

In [ ]:
test['class'].value_counts()

In [ ]:
test[['id','class']].to_csv('submission.csv',index=False)